In [2]:
# 安装库依赖
!pip install scikit-image==0.19.0
!pip install SimpleITK==2.2.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install dicom2nifti==1.0.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install -r requirements.txt

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 57.5 MB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 64.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 56.7 MB/s eta 0:00:01
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
modelarts 1.4.13 requires Pillow<=9.1.1, but you have pillow 9.3.0 which is incompatible.
modelarts 1.4.13 requires psutil~=5.9.1, but you have psutil 5.7.0 which is incompatible.
model

In [ ]:
# 转换数据集
!python src/nnunet/experiment_planning/nnUNet_convert_decathlon_task.py -r ./kits19 -i ./src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task01_kits

In [ ]:
# 数据预处理
!python ./src/nnunet/experiment_planning/nnUNet_plan_and_preprocess.py -t 1  -tl 4 -tf 4 

In [ ]:
# 训练模型
!python train.py 3d_fullres nnUNetTrainerV2_SGD 1 all

In [ ]:
# 产生验证集
!python src/nnunet/generate_testset.py --splits_final=src/nnUNetFrame/DATASET/nnUNet_preprocessed/Task001_kits/splits_final.pkl --fold=0

In [ ]:
# 生成验证集预测
!python eval.py -i src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task001_kits/imagesVal/ -o src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task001_kits/inferVal -t 1 -m 3d_fullres -tr nnUNetTrainerV2_SGD -f all --final_submit=False 

In [ ]:
# 生成验证集评价指标
!python src/nnunet/evaluation/evaluator.py -ref src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task001_kits/labelsVal  -pred src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task001_kits/inferVal -l 0 1 2

In [ ]:
# 生成测试集预测
!python eval.py -i src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task001_kits/imagesTs/ -o src/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task001_kits/inferTs -t 1 -m 3d_fullres -tr nnUNetTrainerV2_SGD -f all --num_threads_preprocessing=8 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020).
If you have questions or suggestions, feel free to open an issue at github nnunet

using model stored in  ./src/nnUNetFrame/DATASET/nnUNet_trained_models/nnUNet/3d_fullres/Task001_kits/nnUNetTrainerV2_SGD__nnUNetPlansv2.1
args.chk model_best
This model expects 1 input modalities for each image
Found 90 unique case ids, here are some examples: ['kits_case_00210' 'kits_case_00269' 'kits_case_00215' 'kits_case_00212'
 'kits_case_00252' 'kits_case_00298' 'kits_case_00228' 'kits_case_00218'
 'kits_case_00260' 'kits_case_00269']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 90
number of cases that still need to be predicted: 90
loading parameters for folds, ['all']
deterministic out
model_best.ckpt.pkl
weight

In [ ]:
# 生成测试集预测的脚本 -i 指定输入文件夹 -o 指定输出文件夹
!python eval.py -i input_folder -o ouput_folder -t 1 -m 3d_fullres -tr nnUNetTrainerV2_SGD -f all --num_threads_preprocessing=8 

In [2]:
!zip -r submit_best.zip *

In [16]:
import moxing as mox
data_obs_url = 'obs://bdci2022/nnUNet-submit/submit_best.zip'
data_local_url = './submit_best.zip'
mox.file.copy_parallel(data_local_url, data_obs_url)

INFO:root:Using MoXing-v2.0.0.rc2.4b57a67b-4b57a67b
INFO:root:Using OBS-Python-SDK-3.20.9.1
